In [ ]:
import numpy as np
import pandas as pd


In [ ]:
#importing data
df=pd.read_excel("/content/drive/My Drive/FLIPR_6/Test_dataset.xlsx",sheet_name="Foreign_Visitors_TS")
City = df['City'].values
df.drop("City",axis=1,inplace=True)

In [ ]:
df.dtypes

Time Series Data    object
Unnamed: 2          object
Unnamed: 3          object
Unnamed: 4          object
Unnamed: 5          object
dtype: object

In [ ]:
df.head()

,Time Series Data,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,April,May,June,July,August
1,1614,1946,2372,2500,2769
2,369179,445559,541045,572860,636502
3,140581,169665,206030,218138,242367
4,2557179,3086249,3747597,3968032,4408916


In [ ]:
df.rename(columns={'Time Series Data':'2020-04-01', 'Unnamed: 2': '2020-05-01', 'Unnamed: 3':'2020-06-01', 'Unnamed: 4':'2020-07-01', 'Unnamed: 5':'2020-08-01'},inplace = True)

In [ ]:
df.drop([0],inplace = True)

In [ ]:
df['2020-04-01'] = df['2020-04-01'].astype(float)
df['2020-05-01'] = df['2020-05-01'].astype(float)
df['2020-06-01'] = df['2020-06-01'].astype(float)
df['2020-07-01'] = df['2020-07-01'].astype(float)
df['2020-08-01'] = df['2020-08-01'].astype(float)

In [ ]:
df.dtypes

2020-04-01    float64
2020-05-01    float64
2020-06-01    float64
2020-07-01    float64
2020-08-01    float64
dtype: object

In [ ]:
df = df.transpose()

In [ ]:
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501
2020-04-01,1614.0,369179.0,140581.0,2557179.0,1800363.0,369179.0,NaN,NaN,NaN,855688.0,863918.0,2557179.0,3317.0,97323.0,2557179.0,369179.0,369179.0,244400.0,244400.0,137963.0,NaN,855688.0,33977.0,2557179.0,855688.0,369179.0,2717138.0,1800363.0,1614.0,NaN,NaN,NaN,855688.0,2557179.0,2557179.0,140581.0,1800363.0,855688.0,244400.0,855688.0,...,61420.0,61420.0,61420.0,855688.0,61420.0,61420.0,61420.0,38851.0,61420.0,137963.0,61420.0,61420.0,61420.0,61420.0,140581.0,61420.0,863918.0,NaN,NaN,NaN,61420.0,61420.0,137963.0,2557179.0,535775.0,61420.0,61420.0,61420.0,61420.0,175816.0,855688.0,61420.0,61420.0,61420.0,61420.0,61420.0,61420.0,61420.0,61420.0,1800363.0
2020-05-01,1946.0,445559.0,169665.0,3086249.0,2172850.0,445559.0,3086249.0,88263.0,445559.0,1032726.0,1042658.0,3086249.0,NaN,117458.0,3086249.0,445559.0,445559.0,294964.0,294964.0,166506.0,445559.0,1032726.0,41006.0,3086249.0,1032726.0,445559.0,3279303.0,2172850.0,1946.0,NaN,NaN,NaN,1032726.0,3086249.0,3086249.0,169665.0,2172850.0,1032726.0,294964.0,1032726.0,...,74125.0,74125.0,74125.0,1032726.0,74125.0,74125.0,74125.0,46888.0,74125.0,166506.0,74125.0,74125.0,74125.0,74125.0,169665.0,74125.0,1042658.0,NaN,NaN,NaN,74125.0,74125.0,166506.0,3086249.0,646624.0,74125.0,74125.0,74125.0,74125.0,212191.0,1032726.0,74125.0,74125.0,74125.0,74125.0,74125.0,74125.0,74125.0,74125.0,2172850.0
2020-06-01,2372.0,541045.0,206030.0,3747597.0,2638469.0,541045.0,3747597.0,107184.0,541045.0,NaN,1266093.0,3747597.0,4867.0,142635.0,3747597.0,541045.0,541045.0,358178.0,NaN,202194.0,541045.0,1254032.0,49801.0,3747597.0,1254032.0,541045.0,3982019.0,2638469.0,2372.0,242245.0,541045.0,1254032.0,1254032.0,3747597.0,3747597.0,206030.0,2638469.0,1254032.0,358178.0,1254032.0,...,90018.0,90018.0,90018.0,1254032.0,90018.0,90018.0,90018.0,56943.0,90018.0,202194.0,90018.0,90018.0,90018.0,90018.0,206030.0,90018.0,1266093.0,NaN,NaN,NaN,90018.0,90018.0,202194.0,3747597.0,785194.0,90018.0,90018.0,90018.0,90018.0,257668.0,1254032.0,90018.0,90018.0,90018.0,90018.0,90018.0,90018.0,90018.0,90018.0,2638469.0
2020-07-01,2500.0,572860.0,218138.0,3968032.0,2793662.0,572860.0,3968032.0,113478.0,572860.0,1327788.0,1340558.0,3968032.0,5143.0,151015.0,3968032.0,572860.0,572860.0,379237.0,NaN,214077.0,572860.0,1327788.0,52719.0,3968032.0,1327788.0,572860.0,4216244.0,2793662.0,2500.0,256484.0,572860.0,1327788.0,1327788.0,3968032.0,3968032.0,218138.0,2793662.0,1327788.0,379237.0,1327788.0,...,95302.0,95302.0,95302.0,1327788.0,95302.0,95302.0,95302.0,60282.0,95302.0,214077.0,95302.0,95302.0,95302.0,95302.0,218138.0,95302.0,1340558.0,NaN,NaN,NaN,95302.0,95302.0,214077.0,3968032.0,831371.0,95302.0,95302.0,95302.0,95302.0,272814.0,1327788.0,95302.0,95302.0,95302.0,95302.0,95302.0,95302.0,95302.0,95302.0,2793662.0
2020-08-01,2769.0,636502.0,242367.0,4408916.0,3104060.0,636502.0,4408916.0,126078.0,636502.0,1475311.0,1489500.0,4408916.0,5705.0,167785.0,4408916.0,636502.0,636502.0,421365.0,421365.0,237854.0,636502.0,1475311.0,58568.0,4408916.0,1475311.0,636502.0,4684707.0,3104060.0,2769.0,284973.0,636502.0,1475311.0,1475311.0,4408916.0,4408916.0,242367.0,3104060.0,1475311.0,421365.0,1475311.0,...,105882.0,105882.0,105882.0,1475311.0,105882.0,105882.0,105882.0,66971.0,105882.0,237854.0,105882.0,105882.0,105882.0,105882.0,242367.0,105882.0,1489500.0,105882.0,636502.0,421365.0,105882.0,105882.0,237854.0,4408916.0,923737.0,105882.0,105882.0,105882.0,105882.0,303118.0,1475311.0,105882.0,105882.0,105882.0,105882.0,105882.0,105882.0,105882.0,105882.0,3104060.0


In [ ]:
for i in range(1,502):
    df.replace(np.NaN,{i:df[i].mean()},inplace=True)

In [ ]:
df.shape

(5, 501)

In [ ]:
df = df.transpose()

In [ ]:
def split_dataset(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix == len(sequence):
            break
        seq_x = sequence[i:end_ix+1]
        X.append(seq_x)
    return pd.DataFrame(X)

In [ ]:
def get_data(data,n_steps):
    database = pd.DataFrame()
    for i in range(len(data)):
        arr = split_dataset(data.values[i],n_steps)
        database = database.append(arr, ignore_index=True)
    print("data:", data.shape)
    print("database:", database.shape)
    return (database)

In [ ]:
n_steps = 1
train_database = get_data(df,n_steps)
Y_train = train_database[n_steps].values
train_database = train_database.drop(n_steps, axis=1)
X_train = train_database.values
print(X_train.shape)
print(Y_train.shape)

data: (501, 5)
database: (2004, 2)
(2004, 1)
(2004,)


In [ ]:
np.savez("/content/drive/My Drive/FLIPR_6/time_data.npz", X_train=X_train, Y_train=Y_train, City= City, n_steps=np.array([n_steps]))

In [ ]:
import numpy as np
data = np.load("/content/drive/My Drive/FLIPR_6/time_data.npz")
X_train = data["X_train"]
Y_train = data["Y_train"]
#City = data['City']
n_steps = data["n_steps"][0]

In [ ]:
print(n_steps)

1


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

In [ ]:
model = Sequential()
model.add(Input((n_steps,1)))
model.add(LSTM(units=100, activation="relu", return_sequences=True))
model.add(LSTM(units=55, activation="relu", return_sequences=True))
model.add(Dense(1))
model.compile(optimizer="adam", loss="msle")

In [ ]:
X = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

In [ ]:
Y = Y_train.reshape(-1,1)

In [ ]:
print("X:", X.shape)
print("Y:", Y.shape)

X: (2004, 1, 1)
Y: (2004, 1)


In [ ]:
model.fit(X,Y,epochs=30, validation_split=0.2)

Epoch 1/30
51/51 [==============================] - 0s 10ms/step - loss: 12.2404 - val_loss: 8.0679
Epoch 2/30
51/51 [==============================] - 0s 4ms/step - loss: 6.2538 - val_loss: 4.7126
Epoch 3/30
51/51 [==============================] - 0s 4ms/step - loss: 3.7857 - val_loss: 2.9477
Epoch 4/30
51/51 [==============================] - 0s 4ms/step - loss: 2.4083 - val_loss: 1.9072
Epoch 5/30
51/51 [==============================] - 0s 4ms/step - loss: 1.5752 - val_loss: 1.2596
Epoch 6/30
51/51 [==============================] - 0s 4ms/step - loss: 1.0478 - val_loss: 0.8422
Epoch 7/30
51/51 [==============================] - 0s 4ms/step - loss: 0.7035 - val_loss: 0.5663
Epoch 8/30
51/51 [==============================] - 0s 3ms/step - loss: 0.4746 - val_loss: 0.3821
Epoch 9/30
51/51 [==============================] - 0s 4ms/step - loss: 0.3210 - val_loss: 0.2579
Epoch 10/30
51/51 [==============================] - 0s 4ms/step - loss: 0.2172 - val_loss: 0.1737
Epoch 11/30
51/51

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 1, 100)            40800     
_________________________________________________________________
lstm_8 (LSTM)                (None, 1, 55)             34320     
_________________________________________________________________
dense_4 (Dense)              (None, 1, 1)              56        
Total params: 75,176
Trainable params: 75,176
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save("/content/drive/My Drive/FLIPR_6/Time_model.h5")

In [ ]:
from tensorflow.keras.models import load_model
time_model = load_model("/content/drive/My Drive/FLIPR_6/Time_model.h5")

In [ ]:
import pandas as pd
test_dataset = pd.read_excel("/content/drive/My Drive/FLIPR_6/Test_dataset.xlsx")
test_fv = test_dataset["Foreign Visitors"]


In [ ]:
input_arr = test_diuresis.values.reshape(-1,1,1)
## n is no of month
n = 1
print(input_arr.shape)

(501, 1, 1)


In [ ]:
for i in range(n):
    input_arr = time_model.predict(input_arr)
    input_arr = input_arr.reshape(-1,1,1)

In [ ]:
input_arr = input_arr.flatten()

In [ ]:
test_dataset["Foreign Visitors"] = input_arr.astype("int64")

In [ ]:
test_dataset.to_excel("/content/drive/My Drive/FLIPR_6/modified_test_dataset.xlsx")